In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
import csv
import os
from tqdm import tqdm
from LeTE import CombinedLeTE

class EventBasedMNIST(Dataset):
    def __init__(self, root, train=True, threshold=0.9, transform=None, download=True):
        self.root = root
        self.train = train
        self.threshold = threshold
        self.transform = transform
        
        self.data = datasets.MNIST(root=self.root, train=self.train, transform=self.transform, download=download)
        self.event_data = []
        self.labels = []
        for img, label in self.data:
            img_flat = img.view(-1)  # (784,)
            events = torch.nonzero(img_flat > self.threshold).squeeze()
            events = torch.sort(events).values
            self.event_data.append(events)
            self.labels.append(label)
    
    def __len__(self):
        return len(self.event_data)
    
    def __getitem__(self, idx):
        return self.event_data[idx], self.labels[idx]

def custom_collate_fn(batch):
    events_list = []
    labels_list = []
    lengths = []
    for events, label in batch:
        events_list.append(events)
        labels_list.append(label)
        lengths.append(events.shape[0])
    labels_tensor = torch.tensor(labels_list, dtype=torch.long)
    padded_events = pad_sequence(events_list, batch_first=True, padding_value=0)  # (batch, max_len)
    lengths = torch.tensor(lengths, dtype=torch.long)
    return padded_events, lengths, labels_tensor

class LSTMClassifier(nn.Module):
    def __init__(self, input_size=784, embedding_dim=32, hidden_dim=128, num_classes=10):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        
        # Init LeTE
        self.time_encoder = CombinedLeTE(embedding_dim, p=0.5)
        
    def forward(self, x, lengths):
        embedded = self.time_encoder(x)  # (batch, seq_len, embedding_dim)
        packed = pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        _, (h_n, c_n) = self.lstm(packed)
        h_n = h_n[-1]  # (batch, hidden_dim)
        out = self.fc(h_n)  # (batch, num_classes)
        return out

def evaluate(model, data_loader, device, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for padded_events, lengths, labels in data_loader:
            padded_events = padded_events.float().to(device)
            lengths = lengths.to(device)
            labels = labels.to(device)
            outputs = model(padded_events, lengths)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * labels.size(0)
            preds = outputs.argmax(dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    avg_loss = total_loss / total_samples
    acc = total_correct / total_samples
    return avg_loss, acc


In [2]:
transform = transforms.ToTensor()
train_dataset = EventBasedMNIST(root="./data", train=True, threshold=0.9, transform=transform, download=True)
test_dataset = EventBasedMNIST(root="./data", train=False, threshold=0.9, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, collate_fn=custom_collate_fn)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:02<00:00, 3549931.79it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 113824.73it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:01<00:00, 1090398.79it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 1763774.54it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMClassifier(input_size=784, embedding_dim=32, hidden_dim=128, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 200
accuracy_per_epoch = []
loss_per_epoch = []
test_accuracy_per_epoch = []
test_loss_per_epoch = []

log_file = "training_log_LeTE.csv"
if os.path.exists(log_file):
    os.remove(log_file)

with open(log_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["epoch", "train_loss", "train_accuracy", "test_loss", "test_accuracy"])

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    # Training
    for batch_idx, (padded_events, lengths, labels) in enumerate(tqdm(train_loader)):
        padded_events = padded_events.float().to(device)
        lengths = lengths.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(padded_events, lengths)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / total_samples
    acc = total_correct / total_samples
    accuracy_per_epoch.append(acc)
    loss_per_epoch.append(avg_loss)

    # Testing
    test_loss, test_acc = evaluate(model, test_loader, device, criterion)
    test_accuracy_per_epoch.append(test_acc)
    test_loss_per_epoch.append(test_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] finished. "
          f"Train Loss: {avg_loss:.4f}, Train Accuracy: {acc:.4f}, "
          f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

    # Save results to csvs
    with open(log_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([epoch+1, avg_loss, acc, test_loss, test_acc])

# Printing results
print("Final Training Accuracy per epoch:", accuracy_per_epoch)
print("Final Training Loss per epoch:", loss_per_epoch)
print("Final Test Accuracy per epoch:", test_accuracy_per_epoch)
print("Final Test Loss per epoch:", test_loss_per_epoch)

  0%|          | 0/118 [00:00<?, ?it/s]

100%|██████████| 118/118 [00:05<00:00, 20.54it/s]


Epoch [1/200] finished. Train Loss: 1.8162, Train Accuracy: 0.3356, Test Loss: 1.5305, Test Accuracy: 0.4351


100%|██████████| 118/118 [00:05<00:00, 21.01it/s]


Epoch [2/200] finished. Train Loss: 1.2690, Train Accuracy: 0.5540, Test Loss: 1.0777, Test Accuracy: 0.6147


...

Epoch [198/200] finished. Train Loss: 0.0009, Train Accuracy: 1.0000, Test Loss: 0.1393, Test Accuracy: 0.9725


100%|██████████| 118/118 [00:05<00:00, 20.90it/s]


Epoch [199/200] finished. Train Loss: 0.0008, Train Accuracy: 1.0000, Test Loss: 0.1419, Test Accuracy: 0.9731


100%|██████████| 118/118 [00:05<00:00, 20.76it/s]


Epoch [200/200] finished. Train Loss: 0.0007, Train Accuracy: 1.0000, Test Loss: 0.1448, Test Accuracy: 0.9729
Final Training Accuracy per epoch: [0.3356, 0.55395, 0.57575, 0.6816166666666666, 0.73725, 0.80175, 0.8441, 0.8728333333333333, 0.89765, 0.8586166666666667, 0.9127166666666666, 0.9210833333333334, 0.93205, 0.9330166666666667, 0.94145, 0.9464666666666667, 0.94585, 0.9533333333333334, 0.9537833333333333, 0.9521666666666667, 0.9571166666666666, 0.9591333333333333, 0.96105, 0.9623833333333334, 0.96335, 0.96555, 0.9665, 0.9670166666666666, 0.9671333333333333, 0.9686, 0.9696333333333333, 0.9698833333333333, 0.9682666666666667, 0.97075, 0.9734166666666667, 0.9739, 0.97275, 0.9746833333333333, 0.9737333333333333, 0.9766166666666667, 0.9769333333333333, 0.9769166666666667, 0.9777333333333333, 0.9773833333333334, 0.9777166666666667, 0.9796, 0.9825, 0.9786, 0.9791666666666666, 0.9811, 0.9814833333333334, 0.9843333333333333, 0.9821, 0.9797666666666667, 0.98195, 0.98405, 0.9854333333333334